In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
from tqdm.auto import tqdm

In [ ]:
import wandb


def get_summary_metrics(sweep_id=None, run_ids=None, run_filter_fn=None):
  api = wandb.Api(timeout=60)
  if sweep_id is not None:
    sweep = api.sweep(sweep_id)
    runs = sweep.runs
  elif run_ids is not None:
    runs = [api.run(rid) for rid in run_ids]
  else:
    raise ValueError

  data = []
  for run in tqdm(runs, desc='Runs', leave=False):
    if callable(run_filter_fn) and not run_filter_fn(run):
      continue
    data.append(dict(run_id=run.id, tags=run.tags, **run.config, **run.summary))

  return pd.DataFrame(data)


In [ ]:
# from llm.datasets import get_dataset_attrs

# columns = [
#     f"{t}/{s}/{m}" for m in ["N", "acc", "ece", "unc_acc", "unc_ece", "qa_unc_ece"]
#     for s in ["val", "test"]
#     for t in get_dataset_attrs("mmlu").get("tasks")]

# eval_7b_run_ids = ["jb3owvcc", "pwdu8g7u", "mzbjwqyj", "luz8ajx5", "bwv5p2mr", "kv4frbky"]

# data = get_summary_metrics(run_ids=[f"deeplearn/llm-calibration/{rid}" for rid in eval_7b_run_ids])[["run_id", "tags", "eval_kshot"] + columns]
# data = data.melt(id_vars=["run_id", "tags", "eval_kshot"], value_vars=columns)
# data["task"] = data.variable.apply(lambda v: v.split("/")[0])
# data["split"] = data.variable.apply(lambda v: v.split("/")[1])
# data["metric"] = data.variable.apply(lambda v: v.split("/")[2])
# data = data.drop(columns=["variable"])

# def _kind(v):
#     if "ft" in v:
#         return "ft"
#     elif "uq" in v:
#         return "uq"
#     return "7b"

# data["kind"] = data.tags.apply(_kind)
# data = data.drop(columns=["tags"])

# data.to_csv("results/mmlu_eval_7b.csv", index=False)

data = pd.read_csv("results/mmlu_eval_7b.csv")

In [ ]:
data = data.pivot(index=['run_id', 'eval_kshot', 'task', 'split', 'kind'], columns=['metric'], values=['value'])
data.columns = [a[-1] for a in data.columns.to_flat_index()]
data = data.reset_index()
data

In [ ]:
data.groupby(["eval_kshot", "split", "kind"])[["acc", "ece", "unc_acc", "qa_unc_ece"]].mean()